<a href="https://colab.research.google.com/github/carlaolivei/Hackathon-Motor-Pegabot/blob/main/New-Model-From-INCT-DD-Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Elaboração de um novo modelo de classificação com base nas informações de usuários avaliados pelo INCT-DD**

In [1]:
#Carrega as bibliotecas
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, matthews_corrcoef, mean_squared_error, r2_score, mean_absolute_percentage_error, max_error, explained_variance_score, median_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
import math
import statistics
import datetime
import pytz
import pickle
## NLTK (biblioteca para processamento de linguagem natural)
import nltk
from nltk.stem.rslp import RSLPStemmer ##http://www.nltk.org/howto/portuguese_en.html

#O primeiro uso exige obter os pacotes adicionais da biblioteca descomentando as linhas a seguir
#Instala os pacotes de termos do nltk (apenas na primeira vez)
#nltk.download()
#nltk.download('rslp')

**O novo modelo de classificação de bots foi construído com base nos usuários manualmente avaliados pelo INCT-DD**

Essa escolha foi tomada considerando que esse conjunto de dados é o melhor que se possui quanto à real possibilidade de um usuário do Twitter ser um bot, não existindo bases de avaliação dentro da realidade brasileira (especialmente quanto ao português), bem como atualizadas

In [3]:
#Busca os dados dos usuários avaliados
datafile_users = "/content/sample_data/inct_users.csv"
df_users = pd.read_csv(datafile_users, header = 0)

#Preenche os valores NaN con 0 apenas para avaliação geral
df_users = df_users.fillna(0)
print(len(df_users))
#Apresenta o total de usuários avaliados
df_users.head()

1074


,Unnamed: 0,error,created_at,default_profile,description,followers_count,friends_count,handle,lang,location,name,profile_image,twitter_id,twitter_is_protected,verified,withheld_in_countries
0,0,0,2009-06-30 01:05:51+00:00,1.0,0,21.0,108.0,lemathes,0.0,"Brasil, São Paulo",Leandro Mathes,http://pbs.twimg.com/profile_images/1141547105...,5.225325e+07,0.0,0.0,[]
1,1,0,2019-03-09 11:29:52+00:00,True,0,4192.0,4886.0,Maurcio98905595,0.0,"MG , Brasil",Maurício Lima,http://pbs.twimg.com/profile_images/1104354755...,1.104344e+18,False,False,[]
2,2,0,2009-10-20 01:19:19+00:00,False,Feliz é a Nação cujo Deus é o Senhor! #ReageBr...,1341.0,1854.0,LunViana,0.0,"Araraquara, Brasil",Luciana,http://pbs.twimg.com/profile_images/1436716357...,8.373752e+07,False,False,[]
3,3,0,2020-05-03 19:06:46+00:00,True,0,2.0,31.0,felipeleixas,0.0,0,Felipe,http://pbs.twimg.com/profile_images/1264366970...,1.257024e+18,False,False,[]
4,4,0,2021-04-25 20:04:17+00:00,True,0,10.0,21.0,JoseCar41451194,0.0,0,Jose Carlos Marques de Albuquerque,http://pbs.twimg.com/profile_images/1429559356...,1.386411e+18,False,False,[]


**No novo modelos são consideradas apenas as informações associadas como "É bot?" de respotas "Sim" ou "Não"**

In [4]:
#Busca a classificação do INCT-DD
datafile_handles = "/content/sample_data/handles_inct.csv" #A classificação é a mesma da sample1
df_handles = pd.read_csv(datafile_handles, header = 0)
print(len(df_handles))
df_handles['É Bot?'].head()

1074


0    não
1    não
2    não
3    sim
4    Não
Name: É Bot?, dtype: object

**As mais recentes postagens dos usuários foram consideradas como um atributo do modelo**

Para a classificação dos usuários, o novo modelo inclui atributos relacionados com as postagens dos usuários, na tentativa de extrair informação mais atualizada e dinâmica de sua atuação. Entretanto, os textos das postagens foram utilizados unificando seus conteúdos e extraindo informações representativas, tais como os termos mais recorrentemente utilizados, diferença no tempo das postagens e repostagens

In [5]:
#Recupera os últimos twittes
datafile_timeline = "/content/sample_data/inct_timelines.csv"
df_timeline = pd.read_csv(datafile_timeline, header = 0)
print(len(df_timeline))
df_timeline.head()

82413


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (7,8,12,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,error,tweet_author,tweet_author_id_str,tweet_contributors,tweet_created_at,tweet_favorite_count,tweet_favorited,tweet_geo,tweet_hashtags,tweet_id,tweet_id_str,tweet_is_retweet,tweet_lang,tweet_place,tweet_retweeted,tweet_source,tweet_text
0,0,NaN,lemathes,52253248,NaN,2022-03-09 02:10:58+00:00,0.0,0.0,NaN,[],1.501380e+18,1501379987747876874,0.0,pt,NaN,0.0,Twitter for Android,@LucianoHangBr Já demorou muito!
1,1,NaN,lemathes,52253248,NaN,2022-03-09 02:10:12+00:00,0.0,False,NaN,[],1.501380e+18,1501379796210757632,False,pt,NaN,False,Twitter for Android,RT @LucianoHangBr: A vida precisa continuar e ...
2,2,NaN,lemathes,52253248,NaN,2022-03-02 21:57:17+00:00,0.0,False,NaN,[],1.499142e+18,1499141820722421760,False,pt,NaN,False,Twitter for Android,Pq ñ mandam uma bomba na cabeça do Pudim e aca...
3,3,NaN,lemathes,52253248,NaN,2022-03-02 16:57:51+00:00,1.0,False,NaN,[],1.499066e+18,1499066467916079105,False,pt,NaN,False,Twitter for Android,"@carteiroreaca Usa máscara, quem quer e acha q..."
4,4,NaN,lemathes,52253248,NaN,2022-03-02 16:54:56+00:00,0.0,False,NaN,[],1.499066e+18,1499065733086695425,False,pt,NaN,False,Twitter for Android,@carteiroreaca Isso aí!!! 👏👏👏👏 Já demorou de m...


Aplica um pré-processamento nos dados para unificar a informação da postagens se tratar de um retweet

In [6]:
#identifica os formatos existentes
df_timeline['tweet_is_retweet'].unique()

array(['0.0', 'False', 'True', False, True], dtype=object)

In [7]:
df_timeline['retweet_tratado'] = df_timeline['tweet_is_retweet'].apply(lambda x: "sim" if (x == 'True' or x == True) else "não")
df_timeline['retweet_tratado'].unique()

array(['não', 'sim'], dtype=object)

In [8]:
#Necessário reverificar no texto do tweet por RT @, pois o campo tweet_is_retweet falha em algumas situações não identificadas
#Parecem ser os RT com comentários adicionais
#for tweet in df_timeline['retweet_tratado', 'tweet_text']:
#    if tweet['retweet_tratado'] == 'não':
#        if tweet['tweet_text'].find("RT @") != -1:
#            tweet['retweet_tratado'] = 'sim'
#len(df_timeline)
#for i in range(len(df_timeline)):
#    if df_timeline.iloc[i]['retweet_tratado'] == 'não':
#        if df_timeline.iloc[i]['tweet_text'].find("RT @") != -1:
#            df_timeline.iloc[i]['retweet_tratado']  = 'sim'
df_timeline['tweet_com_rt_tratado'] = df_timeline['tweet_text'].apply(lambda x: "sim" if x.find("RT @") != -1 else "não" )

In [11]:
#Combina em uma única coluna as informações de retweets e tweets com RT comentados
def reune_rt(retweet,rt):
    if retweet == 'sim' or rt == 'sim':
        return 'sim'
    else:
        return 'não'

df_timeline['retweet_e_tweet_com_rt_tratado'] = df_timeline.apply(lambda x: reune_rt(x.retweet_tratado, x.tweet_com_rt_tratado), axis=1)
df_timeline.head()

,Unnamed: 0,error,tweet_author,tweet_author_id_str,tweet_contributors,tweet_created_at,tweet_favorite_count,tweet_favorited,tweet_geo,tweet_hashtags,...,tweet_id_str,tweet_is_retweet,tweet_lang,tweet_place,tweet_retweeted,tweet_source,tweet_text,retweet_tratado,tweet_com_rt_tratado,retweet_e_tweet_com_rt_tratado
0,0,NaN,lemathes,52253248,NaN,2022-03-09 02:10:58+00:00,0.0,0.0,NaN,[],...,1501379987747876874,0.0,pt,NaN,0.0,Twitter for Android,@LucianoHangBr Já demorou muito!,não,não,não
1,1,NaN,lemathes,52253248,NaN,2022-03-09 02:10:12+00:00,0.0,False,NaN,[],...,1501379796210757632,False,pt,NaN,False,Twitter for Android,RT @LucianoHangBr: A vida precisa continuar e ...,não,sim,sim
2,2,NaN,lemathes,52253248,NaN,2022-03-02 21:57:17+00:00,0.0,False,NaN,[],...,1499141820722421760,False,pt,NaN,False,Twitter for Android,Pq ñ mandam uma bomba na cabeça do Pudim e aca...,não,não,não
3,3,NaN,lemathes,52253248,NaN,2022-03-02 16:57:51+00:00,1.0,False,NaN,[],...,1499066467916079105,False,pt,NaN,False,Twitter for Android,"@carteiroreaca Usa máscara, quem quer e acha q...",não,não,não
4,4,NaN,lemathes,52253248,NaN,2022-03-02 16:54:56+00:00,0.0,False,NaN,[],...,1499065733086695425,False,pt,NaN,False,Twitter for Android,@carteiroreaca Isso aí!!! 👏👏👏👏 Já demorou de m...,não,não,não


In [12]:
df_timeline[df_timeline["retweet_e_tweet_com_rt_tratado"] == 'sim']

,Unnamed: 0,error,tweet_author,tweet_author_id_str,tweet_contributors,tweet_created_at,tweet_favorite_count,tweet_favorited,tweet_geo,tweet_hashtags,...,tweet_id_str,tweet_is_retweet,tweet_lang,tweet_place,tweet_retweeted,tweet_source,tweet_text,retweet_tratado,tweet_com_rt_tratado,retweet_e_tweet_com_rt_tratado
1,1,NaN,lemathes,52253248,NaN,2022-03-09 02:10:12+00:00,0.0,False,NaN,[],...,1501379796210757632,False,pt,NaN,False,Twitter for Android,RT @LucianoHangBr: A vida precisa continuar e ...,não,sim,sim
5,5,NaN,lemathes,52253248,NaN,2022-02-27 13:38:14+00:00,0.0,False,NaN,[],...,1497929065302482946,False,pt,NaN,False,Twitter for Android,"RT @roxmo: Puxa, que pena, passou tão perto!… ...",não,sim,sim
6,6,NaN,lemathes,52253248,NaN,2022-02-18 04:17:53+00:00,0.0,False,NaN,[],...,1494526561902546944,False,pt,NaN,False,Twitter for Android,RT @mila_sayuri: Alguém poderia confirmar se e...,não,sim,sim
7,7,NaN,lemathes,52253248,NaN,2022-02-18 04:11:31+00:00,0.0,False,NaN,[],...,1494524957593845762,False,pt,NaN,False,Twitter for Android,RT @RenzoGracieBJJ: Quando postei aqui o vídeo...,não,sim,sim
8,8,NaN,lemathes,52253248,NaN,2022-02-18 04:10:00+00:00,0.0,False,NaN,[],...,1494524573919940609,False,pt,NaN,False,Twitter for Android,RT @roxmo: Vc confia nas urnas eletrônicas?,não,sim,sim
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82406,82406,NaN,FATIMAC75843178,1349784643244093440,NaN,2022-03-17 12:10:29+00:00,0.0,False,NaN,[],...,1504429966729138176,False,pt,NaN,False,Twitter for Android,RT @EdmarVencedor: @BelaBonoro @OsvaldoLimaJni...,não,sim,sim
82408,82408,NaN,FATIMAC75843178,1349784643244093440,NaN,2022-03-17 12:09:52+00:00,0.0,False,NaN,[],...,1504429810352898052,False,pt,NaN,False,Twitter for Android,RT @BelaBonoro: @OsvaldoLimaJni1 @CeliaSLeao1 ...,não,sim,sim
82409,82409,NaN,FATIMAC75843178,1349784643244093440,NaN,2022-03-17 12:09:18+00:00,0.0,False,NaN,[],...,1504429669613031426,False,pt,NaN,False,Twitter for Android,RT @carlosjordy: Ciro sincero diz de quem é a ...,não,sim,sim
82410,82410,NaN,FATIMAC75843178,1349784643244093440,NaN,2022-03-17 12:08:46+00:00,0.0,False,NaN,[],...,1504429535818924033,False,pt,NaN,False,Twitter for Android,RT @CarlaZambelli38: ATENÇÃO ao alerta do Pres...,não,sim,sim


Extrai a diferença em segundos entre as postagens do usuário

In [13]:
#Incluir uma dedida da distancia temporal entre twittes (mediana e mínimo)
df_handles['Tempo mediano'] = np.array(len(df_handles))
df_handles['Tempo menor']   = np.array(len(df_handles))
iuser = 0
for user in df_handles['handle']:
    df_temp = df_timeline[df_timeline['tweet_author'] == user]
    itweet = 0
    menor = 100000
    difs = list()
    tweet_date_prev = None
    for tweet in df_temp['tweet_created_at']:
        tweet_date = pd.to_datetime(pd.to_datetime(tweet).strftime("%Y-%m-%dT%H:%M:%S.%fZ"))
        if itweet > 0:
            dif = (tweet_date_prev - tweet_date).seconds
            if dif < menor:
                menor = dif
            difs.append(dif)
        else:
            tweet_date_prev = tweet_date
        tweet_date_prev = tweet_date
        itweet += 1
    if len(difs) > 0:
        mediana = statistics.median(difs)
    else:
        mediana = 1000
    print(user + ' - ' + str(menor) + ' - ' + str(mediana)+'\n')
    df_handles['Tempo mediano'][iuser] = mediana
    df_handles['Tempo menor'][iuser]   = menor
    iuser += 1
    
    

lemathes - 16 - 1917

Maurcio98905595 - 1 - 22

LunViana - 2 - 34

felipeleixas - 141 - 40791.0

JoseCar41451194 - 9 - 584

stefmilhori - 0 - 862

Maurio0916 - 11 - 7975

alaincremonezi - 7 - 210

marctrickguedes - 24 - 436



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Valdir_25 - 16 - 10217

HermesMachadoAP - 26 - 3416

euclaudemir - 4 - 8076

LongoMarlongo - 5 - 117

Tadeu88537223 - 4 - 147

EliasBispodeCe1 - 3 - 63

lessa_tadeu - 100000 - 1000

kentyan71 - 4 - 136

Sirenite1 - 1 - 142

elechimamil - 6 - 423

mhelena17 - 4 - 74

victorcalazans - 15 - 405

AnaPedros2308 - 4 - 136

ricardo_lacava - 24 - 235

AslanDeHogwarts - 10 - 1933

Patriota7M - 0 - 24

luGusmao1 - 2 - 15

GilAndrade5 - 6 - 66

ClaytonSampaio5 - 0 - 20900

filhadefridak - 17 - 14945

lucas_neves164 - 100000 - 1000

Thiago48035185 - 6 - 71

herciconti - 0 - 110

FlaviaPauletti - 13 - 2250

ElielAmorim9 - 14 - 207

Lunar_Vante - 6 - 380

lemosmarl - 3 - 10776

joaocarlosjc - 5 - 1533

georgedebarros - 16 - 314

valterpn - 2 - 290

Mariade25585540 - 5 - 249

NTresolavy - 6 - 193

jonny_1309 - 3 - 27605

Dani_BernMor - 16 - 652

lui45807682 - 2 - 180

OluasSnanaj - 22 - 2964

abreumartha - 7 - 104

deuzemaroliveir - 5 - 49

Tahuamello - 100000 - 1000

BorjaoOp - 100000 - 1000

Melchi

**Os dados inicialmente tratados são reunidos com a classificação dada pelo INCT-DD**

In [ ]:
#Reune os dados do usuário com a classificação
df_result_merge = pd.merge(df_handles, df_users, on=['handle'])
print(len(df_result_merge))
df_result_merge.head()

**Os dados das postagens foram reunidos para a extração de informações representativas**

Para viabilizar o treinamento do modelo, os dados por postagens foram convertidos em conjuntos por usuário (autor do tweet, e a representação foi dada por informações sumarizadas ou probabilísticas, por exemplo, as hashtags mais utilizadas ou o percentual de postagens realizadas a partir do Android, iPhone ou Web.

In [15]:
#Reune todos os tweets de um mesmo autor em um único texto, separando apenas por vírgula
df_result_text = df_timeline.groupby('tweet_author').agg({'tweet_text':lambda col: ', '.join(col)}).reset_index()
df_result_text.head()

,tweet_author,tweet_text
0,100_bolsonaro,@OracoesB @wander_fabricio @DinhaCarvalho8 #Bo...
1,13valber1,RT @leandroruschel: Tente encontrar na extrema...
2,1976Mnc,RT @MinEconomia: “Nós estamos assistindo a uma...
3,ACamargo241,RT @juliovschneider: Se liga na viatura daqui ...
4,AControld,"Carro Pajero TR4 4X4 Automatica, podendo sair ..."


In [17]:
#Reune todos as hashtags utilizadas por um mesmo autor em um único texto, separando apenas por vírgula
df_result_hashtags = df_timeline.groupby('tweet_author').agg({'tweet_hashtags':lambda col: ', '.join(col)}).reset_index()
df_result_hashtags.head()

,tweet_author,tweet_hashtags
0,100_bolsonaro,"['Bolsonaro2022'], ['MoroTraidor'], [], ['Moro..."
1,13valber1,"[], [], [], [], [], [], [], [], [], [], [], []..."
2,1976Mnc,"[], [], [], [], [], [], ['PLP235NÃO'], [], ['P..."
3,ACamargo241,"[], [], [], [], [], [], [], [], [], [], [], []..."
4,AControld,"['RedeBBB', 'tbt', 'iphone', 'apple'], ['Natal..."


In [18]:
#Reune a informação de fonte de todos os tweets de um mesmo autor em um único texto, separando apenas por vírgula
df_result_source = df_timeline.groupby('tweet_author').agg({'tweet_source':lambda col: ', '.join(col)}).reset_index()
df_result_source.head()

,tweet_author,tweet_source
0,100_bolsonaro,"Twitter Web App, Twitter Web App, Twitter Web ..."
1,13valber1,"Twitter for Android, Twitter for Android, Twit..."
2,1976Mnc,"Twitter for iPhone, Twitter for iPhone, Twitte..."
3,ACamargo241,"Twitter for Android, Twitter for Android, Twit..."
4,AControld,"Twitter Web App, Twitter Web App, Twitter Web ..."


In [19]:
#Reune as informações de twettes que são retweets
df_result_retweet = df_timeline.groupby('tweet_author').agg({'retweet_tratado':lambda col: ', '.join(col)}).reset_index()
df_result_retweet.head()

,tweet_author,retweet_tratado
0,100_bolsonaro,"não, não, não, não, não, não, não, não, não, n..."
1,13valber1,"não, não, não, não, não, não, não, não, não, n..."
2,1976Mnc,"não, não, não, não, não, não, não, não, não, n..."
3,ACamargo241,"não, não, não, não, não, sim, não, não, não, n..."
4,AControld,"não, não, não, não, não, não, não, não, não, n..."


In [20]:
#Reune as informações de twettes com RT
df_result_tweet_com_rt = df_timeline.groupby('tweet_author').agg({'tweet_com_rt_tratado':lambda col: ', '.join(col)}).reset_index()
df_result_tweet_com_rt.head()

,tweet_author,tweet_com_rt_tratado
0,100_bolsonaro,"não, não, sim, não, não, sim, sim, sim, não, n..."
1,13valber1,"sim, sim, sim, sim, não, não, não, não, não, n..."
2,1976Mnc,"sim, sim, não, não, sim, sim, não, sim, sim, s..."
3,ACamargo241,"sim, sim, sim, sim, sim, não, sim, sim, sim, s..."
4,AControld,"não, não, não, não, não, não, não, não, não, n..."


In [21]:
#Reune as informações da junção de retweets e tweets com rt
df_result_retweet_e_tweet_com_rt = df_timeline.groupby('tweet_author').agg({'retweet_e_tweet_com_rt_tratado':lambda col: ', '.join(col)}).reset_index()
df_result_retweet_e_tweet_com_rt.head()

,tweet_author,retweet_e_tweet_com_rt_tratado
0,100_bolsonaro,"não, não, sim, não, não, sim, sim, sim, não, n..."
1,13valber1,"sim, sim, sim, sim, não, não, não, não, não, n..."
2,1976Mnc,"sim, sim, não, não, sim, sim, não, sim, sim, s..."
3,ACamargo241,"sim, sim, sim, sim, sim, sim, sim, sim, sim, s..."
4,AControld,"não, não, não, não, não, não, não, não, não, n..."


In [22]:
#Reune os dados (merge) do usuários, suas avaliações com texto dos tweets, as hashtags, as fontes e os retweets
df_result_merge = pd.merge(df_handles, df_users, on=['handle'])
df_result_merge = pd.merge(df_result_merge,df_result_text, left_on=['handle'], right_on=['tweet_author'])
df_result_merge = pd.merge(df_result_merge,df_result_hashtags, left_on=['handle'], right_on=['tweet_author'])
df_result_merge = pd.merge(df_result_merge,df_result_source, left_on=['handle'], right_on=['tweet_author'])
df_result_merge = pd.merge(df_result_merge,df_result_retweet, left_on=['handle'], right_on=['tweet_author'])
df_result_merge = pd.merge(df_result_merge,df_result_tweet_com_rt, left_on=['handle'], right_on=['tweet_author'])
df_result_merge = pd.merge(df_result_merge,df_result_retweet_e_tweet_com_rt, left_on=['handle'], right_on=['tweet_author'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'tweet_author_x'} in the result is deprecated and will raise a MergeError in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'tweet_author_x'} in the result is deprecated and will raise a MergeError in a future version.
  


In [23]:
#Exibe parte dos resultados da junção (nem todos os usuários ainda estão ativos e número de amostras diminui)
print(len(df_result_merge))
df_result_merge.head()

834


,Unnamed: 0_x,Unnamed: 0.1,tabelaAmostra,p,É Bot?,"Se você fosse atribuir uma função ao bot, qual seria?",Função #2,Comportamento agressivo?,Comportamento repetitivo com # ou menções?,Parece só Retweetar?,...,tweet_author_y,tweet_hashtags,tweet_author_x,tweet_source,tweet_author_y,retweet_tratado,tweet_author_x,tweet_com_rt_tratado,tweet_author_y,retweet_e_tweet_com_rt_tratado
0,0,1,https://twitter.com/@lemathes,0000.csv,não,não se aplica,NaN,não,não,não,...,lemathes,"[], [], [], [], [], [], [], [], [], [], [], []...",lemathes,"Twitter for Android, Twitter for Android, Twit...",lemathes,"não, não, não, não, não, não, não, não, não, n...",lemathes,"não, sim, não, não, não, sim, sim, sim, sim, s...",lemathes,"não, sim, não, não, não, sim, sim, sim, sim, s..."
1,1,2,https://twitter.com/@Maurcio98905595,0000.csv,não,não se aplica,NaN,não,não,não,...,Maurcio98905595,"[], [], [], [], [], [], [], [], [], [], [], []...",Maurcio98905595,"Twitter for Android, Twitter Web App, Twitter ...",Maurcio98905595,"sim, sim, não, sim, sim, sim, sim, não, sim, s...",Maurcio98905595,"não, não, sim, não, não, sim, não, sim, não, n...",Maurcio98905595,"sim, sim, sim, sim, sim, sim, sim, sim, sim, s..."
2,2,3,https://twitter.com/@LunViana,0000.csv,não,não se aplica,NaN,não,não,não,...,LunViana,"[], [], [], [], [], [], [], [], [], [], [], []...",LunViana,"Twitter for iPhone, Twitter for Android, Twitt...",LunViana,"não, não, não, não, sim, não, não, não, não, n...",LunViana,"sim, sim, sim, sim, não, sim, sim, sim, sim, s...",LunViana,"sim, sim, sim, sim, sim, sim, sim, sim, sim, s..."
3,3,4,https://twitter.com/@felipeleixas,0000.csv,sim,Publicar hashtags,Atacar,sim,sim,não,...,felipeleixas,"[], ['EuApoioVotoImpresso'], [], ['GloboLixo']...",felipeleixas,"Twitter for Android, Twitter for Android, Twit...",felipeleixas,"não, não, não, não, não, não, não, não, não, n...",felipeleixas,"não, não, não, não, sim, não, não, não, não, n...",felipeleixas,"não, não, não, não, sim, não, não, não, não, n..."
4,4,5,https://twitter.com/@JoseCar41451194,0000.csv,Não,não se aplica,NaN,não,não,não,...,JoseCar41451194,"[], [], [], [], [], [], [], [], [], [], ['OsPi...",JoseCar41451194,"Twitter for iPhone, Twitter for iPhone, Twitte...",JoseCar41451194,"não, não, não, não, não, não, não, não, não, n...",JoseCar41451194,"sim, sim, sim, sim, sim, sim, sim, sim, sim, s...",JoseCar41451194,"sim, sim, sim, sim, sim, sim, sim, sim, sim, s..."


**A classificação dos usuários foi padronizada para 0 - Não Bot e 1 - Bot**

In [24]:
#Padroniza a saída da classificação do INCT-DD para bot e monta o conjunto Y
df = df_result_merge
y = df['É Bot?'].apply(lambda x: 1 if (x == 'Sim' or x == 'sim') else 0)
y.reset_index(drop=True, inplace=True)
y.head()

0    0
1    0
2    0
3    1
4    0
Name: É Bot?, dtype: int64

In [ ]:
##Seleciona as colunas para o conjunto X
#feature_cols = ['tweet_text'] #,'tweet_source','tweet_hashtags'
#x = df['tweet_text']
#x.shape

** [Classficando apenas pelo texto dos Twittes (NLTK)] **

In [ ]:
##Prepara o conjunto de dados para treinamento e teste
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1) 

In [25]:
##Método para vetorizar e contabilizar os termos
stemmer = nltk.stem.RSLPStemmer()
class StemmedCountVectorizerRSLPS(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizerRSLPS, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
stemmed_count_vect = StemmedCountVectorizerRSLPS(stop_words=nltk.corpus.stopwords.words('portuguese'))
tfidf_transformer = TfidfTransformer()

LookupError: ignored

In [ ]:
##Pipeline para extrair as informaçoes e classificar com base no texto (pode ser usado ANN ou MNB [MultinomialNB(fit_prior=False)])
#text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect),
#                      ('tfidf', TfidfTransformer()),
#                      ('mnb', MLPClassifier(random_state=1, max_iter=600, activation='relu',solver='adam')),
#])
#text_mnb_stemmed = text_mnb_stemmed.fit(x_train, y_train)

In [ ]:
#text_mnb_stemmed

In [ ]:
##Avalia a classificação
#predicted_mnb_stemmed = text_mnb_stemmed.predict(x_test)
#np.mean(predicted_mnb_stemmed == y_test)

**Os atributos do treinamentos envolvem diversos fatores**

Uma das etapas mais critícas da modelagem é a definição dos atributos que representam o cenário real, nesse sentido foram incluídas o máximo de variáveis que pudessem representar um usuário e suas atividades na rede, desde o tamanho do login escolhido até o tempo mínimo entre suas postagens. Na sequência são realizadas as atividades de extração, tratamento e junção dessas informações como atributos do conjunto de treinamento do modelo.

In [26]:
df.columns #df é o conjunto completo de dados, já com os twittes-hashtags-sources-retweets em campos únicos

Index(['Unnamed: 0_x', 'Unnamed: 0.1', 'tabelaAmostra', 'p', 'É Bot?',
       'Se você fosse atribuir uma função ao bot, qual seria?', 'Função #2',
       'Comportamento agressivo?',
       'Comportamento repetitivo com # ou menções?', 'Parece só Retweetar?',
       'Só compartilha links?', 'Só faz comentários?',
       'Enaltece muito outros usuários?', 'Faz muito uso de emojis?',
       'Tem muitos posts sem textos?', 'Unnamed: 14', 'handle',
       'Tempo mediano', 'Tempo menor', 'Unnamed: 0_y', 'error', 'created_at',
       'default_profile', 'description', 'followers_count', 'friends_count',
       'lang', 'location', 'name', 'profile_image', 'twitter_id',
       'twitter_is_protected', 'verified', 'withheld_in_countries',
       'tweet_author_x', 'tweet_text', 'tweet_author_y', 'tweet_hashtags',
       'tweet_author_x', 'tweet_source', 'tweet_author_y', 'retweet_tratado',
       'tweet_author_x', 'tweet_com_rt_tratado', 'tweet_author_y',
       'retweet_e_tweet_com_rt_tratado'],


In [27]:
df.head()

,Unnamed: 0_x,Unnamed: 0.1,tabelaAmostra,p,É Bot?,"Se você fosse atribuir uma função ao bot, qual seria?",Função #2,Comportamento agressivo?,Comportamento repetitivo com # ou menções?,Parece só Retweetar?,...,tweet_author_y,tweet_hashtags,tweet_author_x,tweet_source,tweet_author_y,retweet_tratado,tweet_author_x,tweet_com_rt_tratado,tweet_author_y,retweet_e_tweet_com_rt_tratado
0,0,1,https://twitter.com/@lemathes,0000.csv,não,não se aplica,NaN,não,não,não,...,lemathes,"[], [], [], [], [], [], [], [], [], [], [], []...",lemathes,"Twitter for Android, Twitter for Android, Twit...",lemathes,"não, não, não, não, não, não, não, não, não, n...",lemathes,"não, sim, não, não, não, sim, sim, sim, sim, s...",lemathes,"não, sim, não, não, não, sim, sim, sim, sim, s..."
1,1,2,https://twitter.com/@Maurcio98905595,0000.csv,não,não se aplica,NaN,não,não,não,...,Maurcio98905595,"[], [], [], [], [], [], [], [], [], [], [], []...",Maurcio98905595,"Twitter for Android, Twitter Web App, Twitter ...",Maurcio98905595,"sim, sim, não, sim, sim, sim, sim, não, sim, s...",Maurcio98905595,"não, não, sim, não, não, sim, não, sim, não, n...",Maurcio98905595,"sim, sim, sim, sim, sim, sim, sim, sim, sim, s..."
2,2,3,https://twitter.com/@LunViana,0000.csv,não,não se aplica,NaN,não,não,não,...,LunViana,"[], [], [], [], [], [], [], [], [], [], [], []...",LunViana,"Twitter for iPhone, Twitter for Android, Twitt...",LunViana,"não, não, não, não, sim, não, não, não, não, n...",LunViana,"sim, sim, sim, sim, não, sim, sim, sim, sim, s...",LunViana,"sim, sim, sim, sim, sim, sim, sim, sim, sim, s..."
3,3,4,https://twitter.com/@felipeleixas,0000.csv,sim,Publicar hashtags,Atacar,sim,sim,não,...,felipeleixas,"[], ['EuApoioVotoImpresso'], [], ['GloboLixo']...",felipeleixas,"Twitter for Android, Twitter for Android, Twit...",felipeleixas,"não, não, não, não, não, não, não, não, não, n...",felipeleixas,"não, não, não, não, sim, não, não, não, não, n...",felipeleixas,"não, não, não, não, sim, não, não, não, não, n..."
4,4,5,https://twitter.com/@JoseCar41451194,0000.csv,Não,não se aplica,NaN,não,não,não,...,JoseCar41451194,"[], [], [], [], [], [], [], [], [], [], ['OsPi...",JoseCar41451194,"Twitter for iPhone, Twitter for iPhone, Twitte...",JoseCar41451194,"não, não, não, não, não, não, não, não, não, n...",JoseCar41451194,"sim, sim, sim, sim, sim, sim, sim, sim, sim, s...",JoseCar41451194,"sim, sim, sim, sim, sim, sim, sim, sim, sim, s..."


De todo os conjuntos de informações disponíveis não foram selecionados aquelas que não poderiam ser automaticamente extraídos dos perfis e atividades dos usuários na rede. Portanto, as classificações como "comportamento agressivo?", "Parece só Retweetar?", entre outras, não foram incluídos no conjunto de treinamento.

In [28]:
feature_cols = ['followers_count', 'friends_count', 'Tempo mediano', 'Tempo menor']
x = df[feature_cols]

In [ ]:
##Converte os testos em frequências
#st = stemmed_count_vect.fit_transform((df['tweet_text']))
#tfidf_transformer = TfidfTransformer()
#x_tfidf = tfidf_transformer.fit_transform(st)
#x_tfidf

In [ ]:
##Inclui as frequências no conjunto x
#x_tfidf.shape
#x.join(pd.DataFrame(x_tfidf.todense()))

In [29]:
len(df['tweet_hashtags'][7].replace("[","").replace("]","").replace(", \'","$").split("$"))
len(df['tweet_hashtags'][7].split(", ["))

100

In [30]:
#Inclui os quantitativos de hashtages utilizadas (e a mediana por postagem)

qtd_hashtags = df['tweet_hashtags'].apply(lambda x: len(x.replace("[","").replace("]","").replace(", \'","$").split("$")))
x['Quantidade hashtags'] = np.array(list(qtd_hashtags))
qtd_hashtags_media = df['tweet_hashtags'].apply(lambda x: len(x.replace("[","").replace("]","").replace(", \'","$").split("$"))/len(x.split(", [")))
x['Quantidade hashtags media'] = np.array(list(qtd_hashtags_media))

x.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,followers_count,friends_count,Tempo mediano,Tempo menor,Quantidade hashtags,Quantidade hashtags media
0,21.0,108.0,1917,16,13,0.130000
1,4192.0,4886.0,22,1,2,0.020000
2,1341.0,1854.0,34,2,6,0.060000
3,2.0,31.0,40791,141,20,0.425532
4,10.0,21.0,584,9,10,0.100000


In [31]:
#Inclui o número de dígitos no nome
username_digitos = df['handle'].apply(lambda x: sum(c.isdigit() for c in str(x)) ) 
x['Digitos no username'] = np.array(list(username_digitos))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [32]:
#O tamanho do nome e do login
tam_username = df['handle'].apply(lambda x: len(str(x)))
tam_nome = df['name'].apply(lambda x: len(str(x)))
x['Tamanho do username'] = np.array(list(tam_username))
x['Tamanho do nome'] = np.array(list(tam_nome))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [33]:
x.head()

,followers_count,friends_count,Tempo mediano,Tempo menor,Quantidade hashtags,Quantidade hashtags media,Digitos no username,Tamanho do username,Tamanho do nome
0,21.0,108.0,1917,16,13,0.130000,0,8,14
1,4192.0,4886.0,22,1,2,0.020000,8,15,13
2,1341.0,1854.0,34,2,6,0.060000,0,8,7
3,2.0,31.0,40791,141,20,0.425532,0,12,6
4,10.0,21.0,584,9,10,0.100000,8,15,34


A fonte do tweet foi considera importante informação, considerando que automações de postagens possam ser facilitadas a partir da versão Web ou que possa existir algum padrão no uso das diferentes fontes. Sendo assim, forneceu-se ao métodos a informação percentual da origem das postagens do mesmo usuário, seja Android, iPhone ou Web.

In [34]:
#Calcula a quantidade de twittes por fontes
fonte_android = df['tweet_source'].apply(lambda x: str(x).count('Twitter for Android') )
fonte_iphone = df['tweet_source'].apply(lambda x: str(x).count('Twitter for iPhone') )
fonte_web = df['tweet_source'].apply(lambda x: str(x).count('Twitter Web App') )

In [35]:
fonte_soma = fonte_android + fonte_iphone + fonte_web
fonte_soma = fonte_soma.apply(lambda x: 1 if x <= 0 else x )

In [36]:
#Calcula o percentual por usuário
fonte_android = fonte_android/fonte_soma
fonte_iphone = fonte_iphone/fonte_soma
fonte_web = fonte_web/fonte_soma

In [37]:
x['Fonte de Android'] = np.array(list(fonte_android))
x['Fonte de iPhone'] = np.array(list(fonte_iphone))
x['Fonte de Web'] = np.array(list(fonte_web))
x = x.fillna(0)
x.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,followers_count,friends_count,Tempo mediano,Tempo menor,Quantidade hashtags,Quantidade hashtags media,Digitos no username,Tamanho do username,Tamanho do nome,Fonte de Android,Fonte de iPhone,Fonte de Web
0,21.0,108.0,1917,16,13,0.130000,0,8,14,1.00,0.00,0.00
1,4192.0,4886.0,22,1,2,0.020000,8,15,13,0.24,0.00,0.76
2,1341.0,1854.0,34,2,6,0.060000,0,8,7,0.18,0.82,0.00
3,2.0,31.0,40791,141,20,0.425532,0,12,6,1.00,0.00,0.00
4,10.0,21.0,584,9,10,0.100000,8,15,34,0.00,1.00,0.00


In [38]:
#Avaliação geral das diferentes fontes
x['Fonte de Android'].describe()

count    834.000000
mean       0.641682
std        0.463189
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: Fonte de Android, dtype: float64

In [39]:
x['Fonte de iPhone'].describe()

count    834.000000
mean       0.198877
std        0.393385
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: Fonte de iPhone, dtype: float64

In [40]:
x['Fonte de Web'].describe()

count    834.000000
mean       0.149848
std        0.330788
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: Fonte de Web, dtype: float64

In [41]:
#Inclui a informação do retweet
df['retweet_tratado'].head()

0    não, não, não, não, não, não, não, não, não, n...
1    sim, sim, não, sim, sim, sim, sim, não, sim, s...
2    não, não, não, não, sim, não, não, não, não, n...
3    não, não, não, não, não, não, não, não, não, n...
4    não, não, não, não, não, não, não, não, não, n...
Name: retweet_tratado, dtype: object

In [42]:
retweet_tratado = df['retweet_tratado'].apply(lambda x: str(x).count('sim')/len(x.split(",")))
x['retweet_tratado_media'] = np.array(list(retweet_tratado))

In [43]:
tweet_com_rt = df['tweet_com_rt_tratado'].apply(lambda x: str(x).count('sim')/len(x.split(",")))
x['tweet_com_rt_tratado_media'] = np.array(list(tweet_com_rt))

In [44]:
retweet_e_tweet_com_rt = df['retweet_e_tweet_com_rt_tratado'].apply(lambda x: str(x).count('sim')/len(x.split(",")))
x['retweet_e_tweet_com_rt_tratado_media'] = np.array(list(retweet_e_tweet_com_rt))

In [45]:
x_novo = x

In [ ]:
##Inclui os textos dos twittes (NLTK)
#st = stemmed_count_vect.fit_transform((df['tweet_text']))
#tfidf_transformer = TfidfTransformer()
#x_tfidf = tfidf_transformer.fit_transform(st)
#x_tfidf
#x_novo = x.join(pd.DataFrame(x_tfidf.todense()))

In [46]:
x_novo.shape

(834, 15)

In [47]:
x_novo.head()

,followers_count,friends_count,Tempo mediano,Tempo menor,Quantidade hashtags,Quantidade hashtags media,Digitos no username,Tamanho do username,Tamanho do nome,Fonte de Android,Fonte de iPhone,Fonte de Web,retweet_tratado_media,tweet_com_rt_tratado_media,retweet_e_tweet_com_rt_tratado_media
0,21.0,108.0,1917,16,13,0.130000,0,8,14,1.00,0.00,0.00,0.10,0.750000,0.840000
1,4192.0,4886.0,22,1,2,0.020000,8,15,13,0.24,0.00,0.76,0.54,0.520000,0.970000
2,1341.0,1854.0,34,2,6,0.060000,0,8,7,0.18,0.82,0.00,0.08,0.840000,0.910000
3,2.0,31.0,40791,141,20,0.425532,0,12,6,1.00,0.00,0.00,0.00,0.042553,0.042553
4,10.0,21.0,584,9,10,0.100000,8,15,34,0.00,1.00,0.00,0.00,0.940000,0.940000


**Com o primeiro conjunto de atributos formado é possível separar o conjunto de dados em treinamento e teste para a elaboração do modelo**

In [48]:
#Cria um modelo de classificação para o conjunto completo
x_train, x_test, y_train, y_test = train_test_split(x_novo, y, test_size=0.3, random_state=1) 

In [49]:
classifier = RandomForestClassifier(n_jobs=3, random_state=1, n_estimators=100)
classifier = classifier.fit(x_train,y_train)
y_pred = classifier.predict(x_test)
np.mean(y_pred == y_test)

0.7330677290836654

In [ ]:
##Seleciona os atributos mais "importantes"
#x_new = SelectKBest(chi2, k=20).fit_transform(x_novo, y)

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(x_new, y, test_size=0.3, random_state=1) 

In [50]:
classifier = RandomForestClassifier(n_jobs=3, random_state=1, n_estimators=100)
classifier = classifier.fit(x_train,y_train)
y_pred = classifier.predict(x_test)
mean = np.mean(y_pred == y_test)
balanced = balanced_accuracy_score(y_test, y_pred)
print ("Mean: " + str(mean) + " | Balanced accuracy: " + str(balanced))
confusion_matrix(y_test, y_pred)

Mean: 0.7330677290836654 | Balanced accuracy: 0.6958582834331337


array([[ 49,  35],
       [ 32, 135]])

In [51]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.60      0.58      0.59        84
           1       0.79      0.81      0.80       167

    accuracy                           0.73       251
   macro avg       0.70      0.70      0.70       251
weighted avg       0.73      0.73      0.73       251



In [52]:
#Classificação com RNA
classifier = MLPClassifier(max_iter=1200, random_state=1, activation='tanh', solver='adam') #activation: logistic, relu, tanh, identity | solver: lbfgs, sgd, adam
classifier = classifier.fit(x_train,y_train)
y_pred = classifier.predict(x_test)
mean = np.mean(y_pred == y_test)
balanced = balanced_accuracy_score(y_test, y_pred)
print ("Mean: " + str(mean) + " | Balanced accuracy: " + str(balanced))

Mean: 0.7250996015936255 | Balanced accuracy: 0.6691616766467066


**Informações de trend topics**

Outra informação que se mostrou de relevância ao longo do trabalho de modelagem foi a relação das postagens de bots com as menções e hashtags listadas nos mais atuais 'trend topics', ou seja, o aparente uso de termos altamente utilizados no momento para possivelmente alavancar a visibilidade da postagem.

Para averiguar essa possibilidade, um sistema de monitoramento dos tópicos mais mencionados foi criado e cada postagem coletada do usuário foi confrontado com os 'trend topics' do período mais próximo. Esse confrontamento gerou um percentual de uso desses tópicos nas postagens dos usuários.

In [54]:
#Busca os dados de todas as trending topics recuperadas
datafile_trends = "/content/sample_data/trends_dataclips_qijpjdyxutqsnrteglrjtwjhdjja.csv"
df_trends = pd.read_csv(datafile_trends, header = 0)
#Preenche os valores NaN con 0 apenas para avaliação geral
df_trends = df_trends.fillna(0)
print(len(df_trends))
df_trends.head()

2680


,trending_id,trend_date_time,trend,user1_id,tweet1,user2_id,tweet2,user3_id,tweet3,user4_id,tweet4,user5_id,tweet5
0,1,2021-12-03 21:03:31.034742,#HappyBirthdayJin,0,-,0,-,0,-,0,-,0,-
1,2,2021-12-03 21:03:31.286371,suga,28431722,Começou!\n\nEles estão todos de terno e sentad...,28431722,Como estão se sentindo com a nova indicação ao...,28431722,Vocês se preocupam com o futuro agora que já r...,78148969,"OH Léo Dias eu vou mandar a fatura pra você, d...",0,-
2,3,2021-12-03 21:03:31.417346,#JINDAY,132699857,"REIZINHO! Jin, membro do BTS, está completando...",0,-,0,-,0,-,0,-
3,4,2021-12-03 21:03:31.527791,#playplusmudo,0,-,0,-,0,-,0,-,0,-
4,5,2021-12-03 21:03:31.720859,TE AMAMOS DAYANE MELLO,34590687,TE AMAMOS DAYANE MELLO ❤️🍷 https://t.co/RcyA8R...,0,-,0,-,0,-,0,-


Entre os passos de tratamentos dos dados das "trend topics" está o ajuste dos padrões de data e hora dos registros, tanto dos tópicos monitorados quanto dos próprios tweets.
A seguir são extraídas as datas dos tweets no formato yyyy-mm-dd, dentro da conversão nos próximos trechos foi também necessário ajustar o "timezone" desses dados.

In [55]:
#Inclui um percentual de trending topics utilizado por tweet
#Para tweet, busca pelos trending topics imediatamente anteriores
df_timeline['Numero de trendings'] = np.array(len(df_timeline))
df_timeline['Numero de trendings'] = 0
df_trends['Trend Date Time Convertido'] = np.array(len(df_trends))

itrend = 0
for x in df_trends['trend_date_time']:
    df_trends['Trend Date Time Convertido'][itrend] = pd.to_datetime(x).strftime("%Y-%m-%d")
    itrend += 1

df_trends.head()   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,trending_id,trend_date_time,trend,user1_id,tweet1,user2_id,tweet2,user3_id,tweet3,user4_id,tweet4,user5_id,tweet5,Trend Date Time Convertido
0,1,2021-12-03 21:03:31.034742,#HappyBirthdayJin,0,-,0,-,0,-,0,-,0,-,2021-12-03
1,2,2021-12-03 21:03:31.286371,suga,28431722,Começou!\n\nEles estão todos de terno e sentad...,28431722,Como estão se sentindo com a nova indicação ao...,28431722,Vocês se preocupam com o futuro agora que já r...,78148969,"OH Léo Dias eu vou mandar a fatura pra você, d...",0,-,2021-12-03
2,3,2021-12-03 21:03:31.417346,#JINDAY,132699857,"REIZINHO! Jin, membro do BTS, está completando...",0,-,0,-,0,-,0,-,2021-12-03
3,4,2021-12-03 21:03:31.527791,#playplusmudo,0,-,0,-,0,-,0,-,0,-,2021-12-03
4,5,2021-12-03 21:03:31.720859,TE AMAMOS DAYANE MELLO,34590687,TE AMAMOS DAYANE MELLO ❤️🍷 https://t.co/RcyA8R...,0,-,0,-,0,-,0,-,2021-12-03


O relacionamento dos trends e dos tweets foi realizado percorrendo todos os trends armazenados para cada tweet em data anterior ao do tweet e, para cada trend nessa condição, verificou-se no texto do tweet a presença de trendings. Caso esteja presente acumulou-se essa ocorrência, finalizando com a ocorrência de uso de uma trend por cada tweet.
Este trecho demanda de melhorias em desempenho e na inclusão de restrições que reduzam o tempo de ocorrência da trend para mais próximo do tweet.

In [56]:
itweet = 0
for tweet in df_timeline['tweet_created_at']:
    tweet_date = pd.to_datetime(pd.to_datetime(tweet).strftime("%Y-%m-%dT%H:%M:%S.%fZ"))
    df_temp = df_trends[df_trends['Trend Date Time Convertido'] == tweet_date.strftime("%Y-%m-%d")] 
        
    itrend = 0
    for trend in df_temp['Trend Date Time Convertido']:
        trend_date = pd.to_datetime(pd.to_datetime(trend).strftime("%Y-%m-%d"))
        if trend_date <= tweet_date.tz_convert(None):
            if df_timeline['tweet_text'][itweet].find(df_trends['trend'][itrend]) != -1: 
                df_timeline['Numero de trendings'][itweet] = df_timeline['Numero de trendings'][itweet] + 1
        itrend += 1
    print(itweet)    
    itweet += 1     

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


A saída de streaming foi truncada nas últimas 5000 linhas.
77413
77414
77415
77416
77417
77418
77419
77420
77421
77422
77423
77424
77425
77426
77427
77428
77429
77430
77431
77432
77433
77434
77435
77436
77437
77438
77439
77440
77441
77442
77443
77444
77445
77446
77447
77448
77449
77450
77451
77452
77453
77454
77455
77456
77457
77458
77459
77460
77461
77462
77463
77464
77465
77466
77467
77468
77469
77470
77471
77472
77473
77474
77475
77476
77477
77478
77479
77480
77481
77482
77483
77484
77485
77486
77487
77488
77489
77490
77491
77492
77493
77494
77495
77496
77497
77498
77499
77500
77501
77502
77503
77504
77505
77506
77507
77508
77509
77510
77511
77512
77513
77514
77515
77516
77517
77518
77519
77520
77521
77522
77523
77524
77525
77526
77527
77528
77529
77530
77531
77532
77533
77534
77535
77536
77537
77538
77539
77540
77541
77542
77543
77544
77545
77546
77547
77548
77549
77550
77551
77552
77553
77554
77555
77556
77557
77558
77559
77560
77561
77562
77563
77564
77565
77566
77567
77568
77569

Para cada tweet foi armazenados o número de trend topics encontrado.

In [ ]:
df_timeline[df_timeline['Numero de trendings'] > 0].describe()
df_timeline['Numero de trendings'].describe()

In [ ]:
df_timeline

As quantidades de trendings utilizadas em cada tweet foram agrupados por autor (usuário), assim foram incluídos na base de treinamento o número de trendings utilizadas, a média de trendings por tweet desse autor e o número máximo de trendings usado em um mesmo tweet.

In [ ]:
#Reune as informações de trends nos tweets por author
df_result_trend = df_timeline.groupby('tweet_author').agg({'Numero de trendings':lambda col: sum(col)/len(col)}).reset_index()
df_result_trend_max = df_timeline.groupby('tweet_author').agg({'Numero de trendings':lambda col: max(col)}).reset_index()
df_result_trend['trends_media'] = df_result_trend['Numero de trendings']
df_result_trend_max['trends_max'] = df_result_trend_max['Numero de trendings']
df_result_trend_max

In [ ]:
df_handles.head()

In [ ]:
df_trends.head()

In [ ]:
trends_unique = df_trends.trend.unique()

In [ ]:
df_result_merge.head()

Os valores referentes aos trendings do usuário são reunidos ("merged") com os dados gerais do usuário

In [ ]:
df_result_merge = pd.merge(df_result_merge,df_result_trend, left_on=['handle'], right_on=['tweet_author'])
df_result_merge = pd.merge(df_result_merge,df_result_trend_max, left_on=['handle'], right_on=['tweet_author'])
df_result_merge

In [ ]:
#df_result_merge_trend = df_result_merge
df_result_merge['qtdtrends'] = np.array(list(tam_username))

ttemp = 0
iuser = 0
for user in df_result_merge.tweet_text:
    for trend in trends_unique:
        if user.find(trend) != -1:
            ttemp = ttemp + 1
    print(str(ttemp) + " - " + str(iuser) + " | " + str((iuser/len(df_result_merge.tweet_text))*100) + "%")
    df_result_merge['qtdtrends'][iuser] = ttemp
    iuser = iuser + 1
    ttemp = 0

In [ ]:
df_result_merge.head()

In [ ]:
x_novo_trend = x_novo

Por fim os dados do monitoramento das trendings são incluídos na base de treinamento.

In [ ]:
x_novo_trend['qtdtrends'] = df_result_merge['qtdtrends']
x_novo_trend['trends_media'] = df_result_merge['trends_media']
x_novo_trend['trends_max'] = df_result_merge['trends_max']

In [ ]:
x_novo_trend.head()

**Conjuntos de treinamento e teste**

Os dados reunidos para geração dos modelos são, então, separados em dados de treinamento e teste para a aplicação dos métodos de aprendizagem de máquina - em especial Random Florest, Redes neuronais artificiais e Gradient Boosting.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_novo_trend, y, test_size=0.3, random_state=1) 

In [ ]:
classifier = RandomForestClassifier(n_jobs=3, random_state=1, n_estimators=100)
classifier = classifier.fit(x_train,y_train)
y_pred = classifier.predict(x_test)
mean = np.mean(y_pred == y_test)
balanced = balanced_accuracy_score(y_test, y_pred)
print ("Mean: " + str(mean) + " | Balanced accuracy: " + str(balanced))
print("Score: " + str(classifier.score(x_test, y_test)))
confusion_matrix(y_test, y_pred)

In [ ]:
classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=1)
classifier = classifier.fit(x_train,y_train)
y_pred = classifier.predict(x_test)
mean = np.mean(y_pred == y_test)
balanced = balanced_accuracy_score(y_test, y_pred)
print ("Mean: " + str(mean) + " | Balanced accuracy: " + str(balanced))
print("Score: " + str(classifier.score(x_test, y_test)))
confusion_matrix(y_test, y_pred)

In [ ]:
importances = classifier.feature_importances_

indices = np.argsort(importances)

fig, ax = plt.subplots(figsize =(10, 6))
ax.barh(range(len(importances)), importances[indices])
ax.set_yticks(range(len(importances)))
_ = ax.set_yticklabels(np.array(x_novo_trend.columns)[indices])

**Resultados**

Os resultados ainda demandam de maior avaliação, especialmente com a variação da semente aleatória para os cortes do conjunto de treinamento e para a aplicação dos métodos. Ainda nesse sentido, demanda-se ainda da seleção de modelos baseada na otimização dos hiperparâmetros dos métodos aplicados.

Mesmo com essas demandas, observa-se uma acurácia aproximada de 74% para os métodos (e aproximadamente 70% ao considerar-se o desbalanceamento da base). Valor considerado bom, dado o complexo cenário tratado. 

Importante ponto a ser destacado que o valor da acurácia baseia-se também em um ponto de corte da consistência da classificação, a qual pode variar en 0.0 e 1.0, valores que atrelam-se à probabilidade da classificação, em que por padrão adota-se o corte em 0.5, apesar da aplicação pode gerar um intervalo mais restrito, deslocando a média/mediana das predições. Dito isso e considerando que não deva ser utilizado apenas o corte "bruto" de bot ou não bot, a associação dessa probabilidade permite melhor compreensão do "risco" do usuário ser efetivamente um bot, bem como permite um deslocamento do rigor dessa classificação. 

Os trechos a seguir avaliam a acurácia considerando a mediana das predições como corte, bem como a comparação dos valores preditos nos grupos de usuários previamente (manualmente) classificados como bot ou não, no qual verifica-se uma clara separação dos valores preditos.

In [ ]:
#x_new_trend = SelectKBest(chi2, k=10).fit_transform(x_novo_trend, y)

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(x_new_trend, y, test_size=0.3, random_state=1) 

In [ ]:
#classifier = RandomForestClassifier(n_jobs=3, random_state=1, n_estimators=100)
#classifier = classifier.fit(x_train,y_train)
#y_pred = classifier.predict(x_test)
#mean = np.mean(y_pred == y_test)
#balanced = balanced_accuracy_score(y_test, y_pred)
#print ("Mean: " + str(mean) + " | Balanced accuracy: " + str(balanced))
#confusion_matrix(y_test, y_pred)

In [ ]:
#x_new_trend

In [ ]:
#confusion_matrix(y_test, y_pred)

In [ ]:
y_pred

In [ ]:
classifier.predict_proba(x_test)

In [ ]:
predicted_proba = classifier.predict_proba(x_test)[0]

In [ ]:
y_test

In [ ]:
np.median(classifier.predict_proba(x_test)[:,1])

In [ ]:
threshold = 0.6
predicted = (classifier.predict_proba(x_test)[:,1] >= threshold).astype(bool)

In [ ]:
np.mean(predicted == y_test)

In [ ]:
x_test_geral = x_test
dtf = [x_test, x_train]
x_test_geral = pd.concat(dtf)

In [ ]:
print(len(x_test_geral))
y_test_temp = y_test
y_test_temp.reset_index(drop=True, inplace=True)
y_test_temp[y_test_temp == 1].index
res_geral = classifier.predict_proba(x_test_geral)[y_test_temp.index,1]
res_sim = classifier.predict_proba(x_test_geral)[y_test_temp[y_test_temp == 1].index,1]
res_nao = classifier.predict_proba(x_test_geral)[y_test_temp[y_test_temp == 0].index,1]

np.median(res_sim)
np.median(res_nao)
bplots = plt.boxplot([res_geral, res_nao, res_sim],  vert = 1, patch_artist = False)

In [ ]:
pd.DataFrame({"Não": res_nao}).describe()

In [ ]:
pd.DataFrame({"Sim": res_sim}).describe()

**Comparação com as predições do Botometer**

Visando a avaliar a qualidade da classificação dos modelos gerados, os mesmos usuários passaram pela avaliação da ferramenta Botometer, já bem conhecida e amplamente utilizada (apesar de sua aplicação com enfoque nas publicações em Inglês).

In [ ]:
#Lê os dados da aplicação do botometer
#Busca os dados dos usuários avaliados
datafile_botometer = "data/handles_inct.csv"
df_botometer = pd.read_csv(datafile_botometer, header = 0)
#Preenche os valores NaN con 0 apenas para avaliação geral
df_botometer = df_botometer.fillna(0)
print(len(df_botometer))
df_botometer.head()

In [ ]:
#Avalia os resultados do botometer
a = len(df_botometer['analise_botometer'])
b = len(df_botometer[(df_botometer['É Bot?'] == 'não') | (df_botometer['É Bot?'] == 'Não')]['analise_botometer'])
c = len(df_botometer[(df_botometer['É Bot?'] == 'sim') | (df_botometer['É Bot?'] == 'Sim')]['analise_botometer'])
print(" " + str(a) + " = " + str(b) + " + " + str(c))
botometer_geral = df_botometer['analise_botometer']
botometer_nao   = df_botometer[(df_botometer['É Bot?'] == 'não') | (df_botometer['É Bot?'] == 'Não')]['analise_botometer']
botometer_sim   = df_botometer[(df_botometer['É Bot?'] == 'sim') | (df_botometer['É Bot?'] == 'Sim')]['analise_botometer']

In [ ]:
plt.figure(figsize =(20, 10)) #(11, 6)
bplots = plt.boxplot([botometer_geral/5, botometer_nao/5, botometer_sim/5, res_geral, res_nao, res_sim],  vert = 1, patch_artist = False)
colors = ['blue', 'green', 'red', 'lightblue', 'lightgreen', 'pink']
c = 0
for i, bplot in enumerate(bplots['boxes']):
    bplot.set(color=colors[c], linewidth=3)
    c += 1
    
colorss = ['blue','blue', 'green', 'green', 'red', 'red', 'lightblue', 'lightblue', 'lightgreen', 'lightgreen', 'pink', 'pink' ]    
c3 = 0
for cap in bplots['caps']:
    cap.set(color=colorss[c3], linewidth=3)
    c3 +=1

plt.title("Boxplot da avaliação do Botometer e do novo modelo Pegabot para os dados avaiados no INCT-DD", loc="center", fontsize=18)
plt.xlabel("Agrupados por: (1) Botometer Geral; (2) Botometer apenas considerados não bots; (3) Botometer apenas considerados bots; (4) Novo Pegabot Geral; (5) Novo Pegabot apenas considerados não bots; (6) Novo Pegabot apenas considerados bots")
plt.ylabel("Avaliação do Botometer")

plt.show()

In [ ]:
import scipy
scipy.stats.kruskal(botometer_geral,  botometer_nao,botometer_sim)

In [ ]:
scipy.stats.kruskal(res_geral,  res_nao,res_sim)